# Skill 5: API Search - Make our bot to talk to any API

We have observed the remarkable synergy created by combining **OpenAI llms with intelligent agents and detailed prompts**. This powerful combination has consistently delivered impressive results. To further capitalize on this capability, we should aim to integrate it with various systems through API communication. Essentially, we can develop within this notebook what is referred to in OpenAI's ChatGPT as 'GPTs.'

Envision a bot that seamlessly integrates with:

- **CRM Systems:** Including Dynamics, Salesforce, and HubSpot.
- **ERP Systems:** Such as SAP, Dynamics, and Oracle.
- **CMS Systems:** Including Adobe, Oracle, and other content management platforms.

The objective is to connect our bot with data repositories, minimizing data duplication as much as possible. These systems typically offer APIs, facilitating programmatic data access.

In this notebook, we plan to develop an agent capable of querying an API to retrieve information and effectively answer questions.
This time we are going to use an open API for currency and digital coins pricing: https://docs.kraken.com/rest/#tag/Market-Data

In [2]:
import os
import json
import requests
from pydantic import BaseModel 

from langchain_openai import AzureChatOpenAI
from langchain_core.tools import BaseTool, StructuredTool
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import RequestsWrapper, TextRequestsWrapper

from langgraph.prebuilt import create_react_agent

from common.utils import num_tokens_from_string, reduce_openapi_spec
from common.prompts import APISEARCH_PROMPT_TEXT

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))


In [3]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [4]:
COMPLETION_TOKENS = 2000

# This notebook needs GPT-4-Turbo (context size of 128k tokens)
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, 
                      streaming=True)

## The Logic

By now, you must infer that the solution for an API Agent has to be something like: give the API specification as part of the system prompt to the LLM , then let a graph agent plan for the right steps to formulate the API calls.<br>

Let's do that. But we must first understand the industry standards of Swagger/OpenAPI


## Introduction to OpenAPI (formerly Swagger)

The OpenAPI Specification, previously known as the Swagger Specification, is a specification for a machine-readable interface definition language for describing, producing, consuming and visualizing web services. Previously part of the Swagger framework, it became a separate project in 2016, overseen by the OpenAPI Initiative, an open-source collaboration project of the Linux Foundation.

OpenAPI Specification is an API description format for REST APIs. An OpenAPI file allows you to describe your entire API, including: Available endpoints (/users for example) and operations on each endpoint ( GET /users, POST /users), description, contact information, license, terms of use and other information.

### Let's get the OpenAPI (Swagger) spec from our desired API that we want to talk to
You can also download it from the Kraken website: https://docs.kraken.com/rest/

Let's see how big is this API specification:

In [5]:
LOCAL_FILE_PATH = "./data/openapi_kraken.json"

In [6]:
# Open and read the JSON file
with open(LOCAL_FILE_PATH, 'r') as file:
    spec = json.load(file)

In [7]:
# You can check the function "reduce_openapi_spec()" in utils.py
reduced_api_spec = reduce_openapi_spec(spec)

In [8]:
api_tokens = num_tokens_from_string(str(spec))
print("API spec size in tokens:",api_tokens)
api_tokens = num_tokens_from_string(str(reduced_api_spec))
print("Reduced API spec size in tokens:",api_tokens)

API spec size in tokens: 66629
Reduced API spec size in tokens: 57398


Sometimes it makes sense to reduce the size of the API Specs by using the `reduce_openapi_spec` function. It's optional.

#### NOTE: As you can see, a large context LLM is needed. `GPT4` or newer models are necessary for this notebook to run succesfully.

### Define Tools

In [9]:
# Most of APIs require Authorization tokens, so we construct the headers using a lightweight python request wrapper called RequestsWrapper
access_token = "ABCDEFG123456" 
headers = {"Authorization": f"Bearer {access_token}"}
requests_wrapper = RequestsWrapper(headers=headers)

toolkit = RequestsToolkit(
    requests_wrapper=RequestsWrapper(headers=headers),
    allow_dangerous_requests=True,
)

In [10]:
toolkit.get_tools()

[RequestsGetTool(requests_wrapper=TextRequestsWrapper(headers={'Authorization': 'Bearer ABCDEFG123456'}, aiosession=None, auth=None, response_content_type='text', verify=True), allow_dangerous_requests=True),
 RequestsPostTool(requests_wrapper=TextRequestsWrapper(headers={'Authorization': 'Bearer ABCDEFG123456'}, aiosession=None, auth=None, response_content_type='text', verify=True), allow_dangerous_requests=True),
 RequestsPatchTool(requests_wrapper=TextRequestsWrapper(headers={'Authorization': 'Bearer ABCDEFG123456'}, aiosession=None, auth=None, response_content_type='text', verify=True), allow_dangerous_requests=True),
 RequestsPutTool(requests_wrapper=TextRequestsWrapper(headers={'Authorization': 'Bearer ABCDEFG123456'}, aiosession=None, auth=None, response_content_type='text', verify=True), allow_dangerous_requests=True),
 RequestsDeleteTool(requests_wrapper=TextRequestsWrapper(headers={'Authorization': 'Bearer ABCDEFG123456'}, aiosession=None, auth=None, response_content_type='te

In [11]:
tools = toolkit.get_tools()

### Define Prompt

In [12]:
printmd(APISEARCH_PROMPT_TEXT)



## Source of Information
- You have access to an API to help answer user queries.
- Here is documentation on the API: {api_spec}

## On how to use the Tools
- You are an agent designed to connect to RestFul APIs.
- Given API documentation above, use the right tools to connect to the API.
- **ALWAYS** before giving the Final Answer, try another method if available. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE Pre-Existing KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- Only use the output of your code to answer the question. 


### Create Graph

In [13]:
graph = create_react_agent(llm, tools, state_modifier=APISEARCH_PROMPT_TEXT.format(api_spec=reduced_api_spec))

### Run the Graph

In [14]:
async def stream_graph_updates_async(graph, user_input: str):
    inputs = {"messages": [("human", user_input)]}

    async for event in graph.astream_events(inputs, version="v2"):
        if (event["event"] == "on_chat_model_stream"):
            # Print the content of the chunk progressively
            print(event["data"]["chunk"].content, end="", flush=True)
        elif (event["event"] == "on_tool_start"  ):
            print("\n--")
            print(f"Calling tool: {event['name']} with inputs: {event['data'].get('input')}")
            print("--")

In [15]:
QUESTION = """
Tell me the price of bitcoin against USD , also the latest OHLC values for Ethereum,
also tell me the bid and ask for Euro.
"""

In [16]:
await stream_graph_updates_async(graph, QUESTION)


--
Calling tool: requests_get with inputs: {'url': 'https://api.kraken.com/0/public/Ticker?pair=XBTUSD'}
--

--
Calling tool: requests_get with inputs: {'url': 'https://api.kraken.com/0/public/OHLC?pair=ETHUSD'}
--

--
Calling tool: requests_get with inputs: {'url': 'https://api.kraken.com/0/public/Ticker?pair=EURUSD'}
--
Here is the requested information:

### 1. **Bitcoin (BTC) against USD:**
   - **Last Trade Price:** $87,661.40
   - **Ask Price:** $87,669.20
   - **Bid Price:** $87,669.10

---

### 2. **Latest OHLC (Open/High/Low/Close) Values for Ethereum (ETH) against USD:**
   - **Open:** $2,189.42
   - **High:** $2,189.81
   - **Low:** $2,187.81
   - **Close:** $2,188.15

---

### 3. **Euro (EUR) against USD:**
   - **Ask Price:** $1.06090
   - **Bid Price:** $1.06088

Let me know if you need further details! 😊

**Great!!** we have now an API Agent, capable of reasoning until it can find the answer given an API documentation.

## Simple APIs

What happens if the API is quite basic, meaning it's just a simple endpoint without a Swagger/OpenAPI definition? Let’s consider the following example:

[CountdownAPI](https://www.countdownapi.com/) is a streamlined version of the eBay API, available as a paid service. We can test it using their demo query, which does not require any Swagger or OpenAPI specification. In this scenario, our main task is to create a tool that retrieves the results. We then pass these results to an agent for analysis, providing answers to user queries, similar to our approach with the Bing Search agent.

In the example below, there is no API specification, but the response from the API is rather lengthy.

In [24]:
# set up the request parameters
params = {
  'x-apikey': 'ca89e780b9986e9d0cdb22ea621556f7b5bd86e0d4ad6ffc1f9bfcef0583c244'

}
headers = {"accept": "application/json",
           'x-apikey': 'ca89e780b9986e9d0cdb22ea621556f7b5bd86e0d4ad6ffc1f9bfcef0583c244'}

# make the http GET request to Countdown API
api_result = requests.get('https://www.virustotal.com/api/v3/ip_addresses/116.203.4.0', headers=headers)

num_tokens = num_tokens_from_string(str(api_result.json())) # this is a custom function we created in common/utils.py
print("Token count:",num_tokens,"\n")  

# print the first 2000 characters of JSON response from Countdown API
print(json.dumps(api_result.json())[:2000], "...")

Token count: 5492 

{"data": {"id": "116.203.4.0", "type": "ip_address", "links": {"self": "https://www.virustotal.com/api/v3/ip_addresses/116.203.4.0"}, "attributes": {"last_https_certificate": {"size": 1059, "public_key": {"algorithm": "EC", "ec": {"oid": "secp256r1", "pub": "3059301306072a8648ce3d020106082a8648ce3d03010703420004f0a67a8c8d13c3069be7c01392c20cebf38553f8b42ce2c0417990882af9558c66eb8d430161d5da5d3538f1ccb3354ebfbfcf4ab4d6ef948f8696eb97135bb0"}}, "thumbprint_sha256": "3556bfff78204b8b3053fabfe2afa22fbba9e95b86e7553d22eb1487d9f632be", "cert_signature": {"signature_algorithm": "sha256RSA", "signature": "66fb79fb4d348f931aa2e14ca87c14be456b085488827ea472a058941cd05612cc72dd61709c41fef8472b7357771c063eaa3626dfa367afd07639c384a46d90a1e8b67078df3fd02dfb7eebf95f76516162341d6dc3c05af73af9d85e60cea0452f79677192b9aa2bd7f77a2ccac57aa750109427891fb746cc857303a6dc37d87dcae41107d297cce2866d15c01e17aceae37f3d0c6d8dd7715302480b37dd24e46c3fdb3d1fd4967d0afbebb2e40573efe0587ef3d3179dad58c6

So, the answer from this product query (the demo only works with 'memory cards' - you will need to sign up for their trial if you want to try any query with an API key), is about ~17k tokens. When combined with the prompt, we won't have any other option than to use GPT-4 models. 

### Define a custom tool to call the API endpoint

In [42]:
class MySimpleAPISearch(BaseTool):
    """Tool for simple API calls that doesn't require OpenAPI 3.0 specs"""
    
    name:str = "apisearch"
    description:str = "useful when the questions includes the term: apisearch.\n"

    api_key: str
    
    def _run(self, query: str) -> str:
        
        params = {
          'api_key': self.api_key,
          'type': 'search',
          'ebay_domain': 'ebay.com',
          'search_term': query
        }
        headers = {"accept": "application/json",
           'x-apikey': 'ca89e780b9986e9d0cdb22ea621556f7b5bd86e0d4ad6ffc1f9bfcef0583c244'}

        # make the http GET request to Countdown API
        api_result = requests.get('https://www.virustotal.com/api/v3/ip_addresses/116.203.4.0', headers=headers)
        
        try:
            response = json.dumps(api_result.json())
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

In [47]:
tools = [MySimpleAPISearch(api_key='demo')]
print(MySimpleAPISearch(api_key='demo'))
graph = create_react_agent(llm, tools, state_modifier=APISEARCH_PROMPT_TEXT.format(api_spec="API provided by the tool"))
print(APISEARCH_PROMPT_TEXT.format(api_spec="API provided by the tool"))

api_key='demo'


## Source of Information
- You have access to an API to help answer user queries.
- Here is documentation on the API: API provided by the tool

## On how to use the Tools
- You are an agent designed to connect to RestFul APIs.
- Given API documentation above, use the right tools to connect to the API.
- **ALWAYS** before giving the Final Answer, try another method if available. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE Pre-Existing KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- Only use the output of your code to answer the question. 



This time let's use the .stream() method 

In [37]:
QUESTION = 'Give me information relevant about the ip provided, what is the score? ISP? is it on black list? wich? At the end make and evaluation and provide a veredict if the ip is malicious or not'

In [38]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
inputs = {"messages": [("user", QUESTION)]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Give me information relevant about the ip provided, what is the score? ISP? is it on black list? wich? At the end make and evaluation and provide a veredict if the ip is malicious or not
================================== Ai Message ==================================
Tool Calls:
  apisearch (call_D2isShzwTeA923lYKAQeBDs1)
 Call ID: call_D2isShzwTeA923lYKAQeBDs1
  Args:
    query: IP information, score, ISP, blacklist status
================================= Tool Message =================================
Name: apisearch

{"data": {"id": "116.203.4.0", "type": "ip_address", "links": {"self": "https://www.virustotal.com/api/v3/ip_addresses/116.203.4.0"}, "attributes": {"jarm": "40d1db40d0000001dc43d1db1db43d2f70423af611c314ea6b67982756f7de", "network": "116.202.0.0/15", "last_analysis_results": {"Acronis": {"method": "blacklist", "engine_name": "Acronis", "category": "harmless", "result": "clean"}, "0xSI_f33

# Summary

In this notebook, we learned about how to create very smart API agents for simple or complex APIs that use Swagger or OpenAPI specifications.
We see, again, that the key to success is to use: Agents with Expert tools + GPT-4 + good prompts.

As homework, try to create a shopping assistant for Etsy e-commerce site using the following API spec: (you will need to register for free and create an API-Key)

- https://developers.etsy.com/documentation/
- https://www.etsy.com/openapi/generated/oas/3.0.0.json

# NEXT

The Next Notebook will guide you on how to add vision and audio to our engine.